## Problem 4.2 - Error Decomposition (by computer)

In [1]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt

##### functions for approximation and estimation error

In [17]:
# define function for calculation the approximation error
def approximation_error(N):
    max_k = (N - 1) // 2
    return 1/4 - 2/(np.pi)**2 * sum([1/(2 * k + 1)**2 for k in range(1,max_k + 1)])

In [ ]:
# define function for the estimation error depending on mu and lambda
def estimation_error(mu, l):
    if (len(mu) != len(l)):
        print("dimension mismatch between mu and lambda")
        return
    return sum((m - l)**2 for m, l in zip(mu, l))

##### approximation error 

In [ ]:
# plot the approximation error for 1 <= N <= 10000 (logarimthmically in x and y)
N_values = range(1, 10001)
approx_errors = [approximation_error(N) for N in N_values]

plt.figure(figsize=(10, 6))
plt.loglog(N_values, approx_errors)
plt.xlabel('N')
plt.ylabel('Approximation Error')
plt.title('Approximation Error vs N')
plt.grid(True)
plt.show()

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [ ]:
# fix sample size and calculate estimation error
n = 1000
model_class_complexities = np.floor(np.power(10, np.linspace(0, 3, 20)))
est_error = []

for N in model_class_complexities:
    k = (N - 1) // 2
    indk = np.arange(0, k+1)
    lam = np.power(-1, indk) * np.sqrt(2)/np.pi / (2*indk + 1)
    lam = np.append([.5], np.ravel([lam, np.zeros(k+1)], 'F'))
    lam = lam[:-1]  # remove last element to match size N

    # average estimation error over 10 trials
    trials = 10
    total_error = 0
    for _ in range(trials):
        X = np.random.normal(0, 0.1, n)
        true_y = X @ lam
        mu = np.linalg.pinv(X.T @ X) @ X.T @ true_y
        total_error += estimation_error(mu, lam)
    est_error.append(total_error / trials)

# plot estimation error against model class complexity
plt.figure(figsize=(10, 6))
plt.loglog(model_class_complexities, est_error)
plt.xlabel('Model Class Complexity (N)')
plt.ylabel('Estimation Error')
plt.title('Estimation Error vs Model Class Complexity')
plt.grid(True)
plt.show()


TypeError: expected a sequence of integers or a single integer, got 'np.float64(1.0)'